In [1]:
import os
import numpy as np 
import re
import io
from utilities.features_csv import to_csv
import pandas as pd
from IPython.display import display

# Preprocessing

In [2]:
import re 
import csv 
docs = {}
labels = {}
#I only work if you run me in the same folder as the actual text files 

with open('training_labels.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            id = int(row[1])
            score= float(row[2])
            labels[id]=score
            line_count += 1
i = 0
for root, dirs, files in os.walk("corpus/"):
    for file in files:
        if file.endswith(".txt"):
            id = re.sub("[^0-9]", "", file)
            id = int(id) 
            path_file = os.path.join(root,file)
            curdir = path_file
            file = open(curdir, 'r', errors='ignore')
            i+=1
            text = file.read()
            docs[id] = text

        

In [3]:
to_csv(docs, labels)

In [3]:
data = pd.read_csv("final.csv")
display(data.head(n=10))

,iD,minor,geo-location,contact_email,vendors,sell_personal,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,Score
0,20481,1,0,0,1,0,0,0,75.245659,183.114146,-4.42,9.08,3.0
1,20484,0,0,1,0,0,1,1,58.618557,141.546392,37.79,7.02,3.0
2,854,0,0,0,0,0,0,0,55.264865,133.162162,46.30,6.60,2.0
3,20490,1,0,1,0,0,0,1,70.405195,171.012987,7.88,8.48,3.0
4,20493,0,0,0,0,0,0,0,81.172242,197.930605,-10.99,9.82,2.0
5,173,0,0,1,0,0,0,1,47.731507,114.328767,65.41,5.67,3.0
6,11096,0,0,1,0,0,0,0,49.727168,119.317919,60.35,5.92,2.0
7,20500,0,0,0,0,0,0,1,92.804233,227.010582,-48.97,11.26,2.0
8,516,0,1,0,0,0,0,1,66.037158,160.092896,18.96,7.94,2.0
9,20506,0,0,0,0,0,0,0,42.903529,102.258824,77.66,5.07,2.0


In [4]:
#Split the data into labels and features
scores = data['Score']

features_raw = data.drop(['Score', 'iD', 'vendors'] , axis = 1)
# features_raw = data.drop("vendors", axis =1)

In [5]:
from sklearn.preprocessing import MinMaxScaler
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()
numerical = ['fog_index', 'avg_sentence_length', 'flesch_reading_ease', 'dale_chall_readability_score']
features_log_minmax_transform = pd.DataFrame(data = features_raw)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_raw[numerical])


# Show an example of a record with scaling applied
display(features_log_minmax_transform.head(n = 5))

,minor,geo-location,contact_email,sell_personal,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score
0,1,0,0,0,0,0,0.660803,0.660803,0.332320,0.660400
1,0,0,1,0,1,1,0.418325,0.418325,0.569989,0.418331
2,0,0,0,0,0,0,0.369417,0.369417,0.617905,0.368978
3,1,0,1,0,0,1,0.590213,0.590213,0.401577,0.589894
4,0,0,0,0,0,0,0.747232,0.747232,0.295327,0.747356


In [6]:
# Import train_test_split
from sklearn.model_selection import train_test_split
# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_log_minmax_transform, 
                                                    scores, 
                                                    test_size = 0.2, 
                                                    random_state = 0)
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 347 samples.
Testing set has 87 samples.


# Random Forrest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

forest = RandomForestClassifier(n_estimators=50, max_depth=None,random_state=1, criterion="entropy")

forest.fit(X_train, y_train)

forrest_predict = forest.predict(X_test)
f1_score(y_test, forrest_predict, average='macro')

0.8799399399399398

In [15]:
from sklearn.metrics import classification_report
target_names = ['1', '2', '3', '4', '5']
print(classification_report(y_test, forrest_predict, target_names=target_names))

              precision    recall  f1-score   support

           1       0.75      0.94      0.83        16
           2       0.85      0.89      0.87        44
           3       0.94      0.71      0.81        21
           4       1.00      0.80      0.89         5
           5       1.00      1.00      1.00         1

   micro avg       0.85      0.85      0.85        87
   macro avg       0.91      0.87      0.88        87
weighted avg       0.86      0.85      0.85        87



# Extra Decision Trees

In [17]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier


extra = ExtraTreesClassifier(n_estimators = 5, random_state = 14, criterion='entropy')
extra.fit(X_train, y_train)

predictions_extra = extra.predict(X_test)
f1_score(y_test, predictions_extra, average='micro')

0.8045977011494253

In [22]:
print(classification_report(y_test, predictions_extra, target_names=target_names))

              precision    recall  f1-score   support

           1       0.71      0.94      0.81        16
           2       0.84      0.82      0.83        44
           3       0.82      0.67      0.74        21
           4       0.80      0.80      0.80         5
           5       1.00      1.00      1.00         1

   micro avg       0.80      0.80      0.80        87
   macro avg       0.84      0.84      0.84        87
weighted avg       0.81      0.80      0.80        87



# AdaBoost

In [16]:
from sklearn.ensemble import AdaBoostClassifier
tree = DecisionTreeClassifier(max_depth=10, criterion='gini', min_samples_split=5)
ada = AdaBoostClassifier(base_estimator=extra, 
                         n_estimators=50, learning_rate=.2,
                         algorithm='SAMME.R', random_state=4)
ada.fit(X_train, y_train)
predictions_ada = ada.predict(X_test)
f1_score(y_test, predictions_ada, average='micro')

NameError: name 'DecisionTreeClassifier' is not defined

In [12]:
print(classification_report(y_test, predictions_ada, target_names=target_names))

              precision    recall  f1-score   support

           1       0.90      0.56      0.69        16
           2       0.82      0.84      0.83        44
           3       0.62      0.76      0.68        21
           4       0.80      0.80      0.80         5
           5       1.00      1.00      1.00         1

   micro avg       0.77      0.77      0.77        87
   macro avg       0.83      0.79      0.80        87
weighted avg       0.79      0.77      0.77        87



In [12]:
from sklearn import model_selection
X = features_log_minmax_transform
y = scores
num_folds = 5
num_instances = len(X)
loocv = model_selection.LeaveOneOut()
model = ExtraTreesClassifier(n_estimators=200, max_depth=11,random_state=None, 
                             criterion="gini", min_samples_split=5)
results = model_selection.cross_val_score(model, X, y, cv=loocv)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Accuracy: 80.415% (39.686%)
